# BRAINWORKS - Relative Citation Amounts
[Mohammad M. Ghassemi](https://ghassemi.xyz), DATA Scholar, 2021

<hr>

## 0. Install Dependencies:
To begin, please import the following external and internal python libraries

In [57]:
# External Libraries
from   pprint import pprint

# Inernal Libraries
from configuration.config import config
from utils.database.database import database   # import the utility
db = database()   

## 1. Extract a set of papers and a reference group:

In [58]:
core_papers = db.query(
                """
                WITH u19_pis AS(
                SELECT pi_names, 
                       pi_ids, 
                       core_project_num, 
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start
                       FROM projects
                       where core_project_num IN ('U19NS104590','U19NS104655','U19NS104648','U19NS104653','U19NS104649', 
                                                  'U19NS112953','U19NS113201','U19NS112959', 'U19NS118246', 'U19NS123716','U19NS123719')
                       -- AND pi_names not like '%(contact)%'
                       GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )

                ,positive_project_info AS (
                SELECT DISTINCT 
                       u19_pis.budget_start AS 'U19',
                       projects.pi_names,
                       projects.pi_ids,
                       projects.activity,
                       projects.foa_number,
                       projects.core_project_num,
                       SUM(projects.total_cost) as total_cost,
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start,
                       MAX(YEAR(STR_TO_DATE(projects.budget_end, '%m/%d/%Y') )) as budget_end
                FROM projects
                LEFT JOIN u19_pis on u19_pis.pi_ids = projects.pi_ids
                       WHERE (projects.core_project_num IN (SELECT core_project_num FROM u19_pis))
                         and projects.core_project_num LIKE 'U19NS%'
                         -- AND projects.pi_names NOT LIKE '%(contact)%'
                GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )

                ,negative_project_info AS (
                SELECT DISTINCT 
                       u19_pis.budget_start AS 'U19',
                       projects.pi_names,
                       projects.pi_ids,
                       projects.activity,
                       projects.foa_number,
                       projects.core_project_num,
                       SUM(projects.total_cost) as total_cost,
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start,
                       MAX(YEAR(STR_TO_DATE(projects.budget_end, '%m/%d/%Y') )) as budget_end
                FROM projects
                LEFT JOIN u19_pis on u19_pis.pi_ids = projects.pi_ids 
                       WHERE projects.pi_ids IN (SELECT pi_ids FROM positive_project_info)
                       and projects.core_project_num NOT LIKE 'U19NS%'
                       AND projects.year >= 2015 

                GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )

                ,complete_project_info AS(
                SELECT * FROM negative_project_info
                UNION ALL
                SELECT * FROM positive_project_info
                )

                ,pi_names as(
                select distinct lower(TRIM(SUBSTRING_INDEX(pi_names,',',1))) as last_name,
                       lower(SUBSTRING_INDEX(TRIM(REPLACE(SUBSTRING_INDEX(pi_names,',',-1),';','')),' ',1)) as first_name 
                       from complete_project_info
                )

                ,papers_with_topics AS(
                SELECT DISTINCT
                       CASE WHEN pi_names.last_name IS NOT NULL THEN 1 ELSE 0 END AS is_pi,
                       budget_start,
                       complete_project_info.U19,
                       link_tables.pmid,
                       CONCAT(CONCAT(LEFT(affiliations.first_name,1),' '), affiliations.last_name) as name_key,
                       affiliations.first_name, 
                       affiliations.middle_name, 
                       affiliations.last_name, 
                       publications.pub_date,
                       publications.pub_date >= '2017-01-01' AS u19_period
                FROM complete_project_info
                JOIN link_tables   ON link_tables.project_number = complete_project_info.core_project_num
                JOIN publications  ON publications.pmid          = link_tables.pmid 
                JOIN affiliations  ON affiliations.pmid          = link_tables.pmid
                LEFT JOIN pi_names ON LEFT(pi_names.first_name,1) = LEFT(affiliations.first_name,1) and pi_names.last_name = affiliations.last_name
                WHERE publications.pub_date >= '2013-01-01' AND publications.pub_date <= '2020-12-31'
                ORDER BY U19 DESC, pub_date
                )

                ,subset as(
                SELECT *
                  FROM topics 
                 WHERE class = 'major'
                 AND pmid IN (SELECT DISTINCT pmid FROM papers_with_topics)
                )

                ,number_of_topics as(
                SELECT pmid, COUNT(pmid) as num_topics FROM subset  GROUP BY pmid
                )

                ,shared_topics AS(
                SELECT s.*, t.pmid as other_pmid, n.num_topics
                FROM subset s
                left join topics t ON t.topic_id = s.topic_id and t.pmid != s.pmid
                left join number_of_topics n on n.pmid = s.pmid
                WHERE (YEAR(t.pub_date) = YEAR(s.pub_date)) AND (MONTH(t.pub_date) = MONTH(s.pub_date)) -- MODIFY
                AND t.class = 'major'
                )

                ,pmid_list AS(
                SELECT DISTINCT pmid from shared_topics
                )
                SELECT * FROM pmid_list
                """)

In [59]:
comparison_papers = db.query(
                """
                WITH u19_pis AS(
                SELECT pi_names, 
                       pi_ids, 
                       core_project_num, 
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start
                       FROM projects
                       where core_project_num IN ('U19NS104590','U19NS104655','U19NS104648','U19NS104653','U19NS104649', 
                                                  'U19NS112953','U19NS113201','U19NS112959', 'U19NS118246', 'U19NS123716','U19NS123719')
                       -- AND pi_names not like '%(contact)%'
                       GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )
                -- SELECT * FROM u19_pis;

                 -- SELECT U19 PIS THAT recieved the award in 2017, and 2020
                ,positive_project_info AS (
                SELECT DISTINCT 
                       u19_pis.budget_start AS 'U19',
                       projects.pi_names,
                       projects.pi_ids,
                       projects.activity,
                       projects.foa_number,
                       projects.core_project_num,
                       SUM(projects.total_cost) as total_cost,
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start,
                       MAX(YEAR(STR_TO_DATE(projects.budget_end, '%m/%d/%Y') )) as budget_end
                FROM projects
                LEFT JOIN u19_pis on u19_pis.pi_ids = projects.pi_ids
                       WHERE (projects.core_project_num IN (SELECT core_project_num FROM u19_pis))
                         and projects.core_project_num LIKE 'U19NS%'
                         -- AND projects.pi_names NOT LIKE '%(contact)%'
                GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )
                -- SELECT * FROM positive_project_info;

                 -- FOR all PIs that recieved U19s in 2017 and 2020, find all other grants they recieved that were not u19 after 2010
                ,negative_project_info AS (
                SELECT DISTINCT 
                       u19_pis.budget_start AS 'U19',
                       projects.pi_names,
                       projects.pi_ids,
                       projects.activity,
                       projects.foa_number,
                       projects.core_project_num,
                       SUM(projects.total_cost) as total_cost,
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start,
                       MAX(YEAR(STR_TO_DATE(projects.budget_end, '%m/%d/%Y') )) as budget_end
                FROM projects
                LEFT JOIN u19_pis on u19_pis.pi_ids = projects.pi_ids 
                       WHERE projects.pi_ids IN (SELECT pi_ids FROM positive_project_info)
                       and projects.core_project_num NOT LIKE 'U19NS%'
                       -- AND projects.pi_names not like '%(contact)%'
                       AND projects.year >= 2015 

                GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )
                -- SELECT * FROM negative_project_info;

                -- Combine u19 and non-u19 grants 
                ,complete_project_info AS(
                SELECT * FROM negative_project_info
                UNION ALL
                SELECT * FROM positive_project_info
                )
                -- SELECT * FROM complete_project_info;

                -- Parse the PI names into first, and last
                ,pi_names as(
                select distinct lower(TRIM(SUBSTRING_INDEX(pi_names,',',1))) as last_name,
                       lower(SUBSTRING_INDEX(TRIM(REPLACE(SUBSTRING_INDEX(pi_names,',',-1),';','')),' ',1)) as first_name 
                       from complete_project_info
                )
                -- SELECT * FROM pi_names;

                -- KEEP ONLY PAPERS FROM 2015 - 2019; denote all papers post 2017 as u19_period 
                ,papers_with_topics AS(
                SELECT DISTINCT
                       CASE WHEN pi_names.last_name IS NOT NULL THEN 1 ELSE 0 END AS is_pi,
                       budget_start,
                       complete_project_info.U19,
                       link_tables.pmid,
                       CONCAT(CONCAT(LEFT(affiliations.first_name,1),' '), affiliations.last_name) as name_key,
                       affiliations.first_name, 
                       affiliations.middle_name, 
                       affiliations.last_name, 
                       publications.pub_date,
                       publications.pub_date >= '2017-01-01' AS u19_period
                FROM complete_project_info
                JOIN link_tables   ON link_tables.project_number = complete_project_info.core_project_num
                JOIN publications  ON publications.pmid          = link_tables.pmid 
                JOIN affiliations  ON affiliations.pmid          = link_tables.pmid
                LEFT JOIN pi_names ON LEFT(pi_names.first_name,1) = LEFT(affiliations.first_name,1) and pi_names.last_name = affiliations.last_name
                WHERE publications.pub_date >= '2013-01-01' AND publications.pub_date <= '2020-12-31'
                ORDER BY U19 DESC, pub_date
                )

                -- SELECT * FROM papers_with_topics;
                ,subset as(
                SELECT *
                  FROM topics 
                 WHERE class = 'major'
                 AND pmid IN (SELECT DISTINCT pmid FROM papers_with_topics)
                )

                -- SELECT * FROM subset;
                ,number_of_topics as(
                SELECT pmid, COUNT(pmid) as num_topics FROM subset  GROUP BY pmid
                )

                -- SELECT * FROM number_of_topics;
                ,shared_topics AS(
                SELECT s.*, t.pmid as other_pmid, n.num_topics
                FROM subset s
                left join topics t ON t.topic_id = s.topic_id and t.pmid != s.pmid
                left join number_of_topics n on n.pmid = s.pmid
                WHERE (YEAR(t.pub_date) = YEAR(s.pub_date)) AND (MONTH(t.pub_date) = MONTH(s.pub_date)) -- MODIFY
                AND t.class = 'major'
                )

                -- SELECT * FROM shared_topics;
                -- We're using this to collect 
                ,pmid_list AS(
                SELECT DISTINCT other_pmid from shared_topics
                )
                SELECT * FROM pmid_list
                """)

In [60]:
core  = ','.join([str(x['pmid']) for x in core_papers])
other = ','.join([str(x['other_pmid']) for x in comparison_papers])

In [61]:
stats = db.query(
f"""
WITH u19_pis AS(
                SELECT pi_names, 
                       pi_ids, 
                       core_project_num, 
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start
                       FROM projects
                       where core_project_num IN ('U19NS104590','U19NS104655','U19NS104648','U19NS104653','U19NS104649', 
                                                  'U19NS112953','U19NS113201','U19NS112959', 'U19NS118246', 'U19NS123716','U19NS123719')
                       -- AND pi_names not like '%(contact)%'
                       GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )
                -- SELECT * FROM u19_pis;

                 -- SELECT U19 PIS THAT recieved the award in 2017, and 2020
                ,positive_project_info AS (
                SELECT DISTINCT 
                       u19_pis.budget_start AS 'U19',
                       projects.pi_names,
                       projects.pi_ids,
                       projects.activity,
                       projects.foa_number,
                       projects.core_project_num,
                       SUM(projects.total_cost) as total_cost,
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start,
                       MAX(YEAR(STR_TO_DATE(projects.budget_end, '%m/%d/%Y') )) as budget_end
                FROM projects
                LEFT JOIN u19_pis on u19_pis.pi_ids = projects.pi_ids
                       WHERE (projects.core_project_num IN (SELECT core_project_num FROM u19_pis))
                         and projects.core_project_num LIKE 'U19NS%'
                         -- AND projects.pi_names NOT LIKE '%(contact)%'
                GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )
                -- SELECT * FROM positive_project_info;

                 -- FOR all PIs that recieved U19s in 2017 and 2020, find all other grants they recieved that were not u19 after 2010
                ,negative_project_info AS (
                SELECT DISTINCT 
                       u19_pis.budget_start AS 'U19',
                       projects.pi_names,
                       projects.pi_ids,
                       projects.activity,
                       projects.foa_number,
                       projects.core_project_num,
                       SUM(projects.total_cost) as total_cost,
                       MIN(YEAR(STR_TO_DATE(projects.budget_start, '%m/%d/%Y'))) as budget_start,
                       MAX(YEAR(STR_TO_DATE(projects.budget_end, '%m/%d/%Y') )) as budget_end
                FROM projects
                LEFT JOIN u19_pis on u19_pis.pi_ids = projects.pi_ids 
                       WHERE projects.pi_ids IN (SELECT pi_ids FROM positive_project_info)
                       and projects.core_project_num NOT LIKE 'U19NS%'
                       -- AND projects.pi_names not like '%(contact)%'
                       AND projects.year >= 2015 

                GROUP BY projects.pi_names, projects.activity, projects.core_project_num
                )
                -- SELECT * FROM negative_project_info;

                -- Combine u19 and non-u19 grants 
                ,complete_project_info AS(
                SELECT * FROM negative_project_info
                UNION ALL
                SELECT * FROM positive_project_info
                )
                -- SELECT * FROM complete_project_info;

                -- Parse the PI names into first, and last
                ,pi_names as(
                select distinct lower(TRIM(SUBSTRING_INDEX(pi_names,',',1))) as last_name,
                       lower(SUBSTRING_INDEX(TRIM(REPLACE(SUBSTRING_INDEX(pi_names,',',-1),';','')),' ',1)) as first_name 
                       from complete_project_info
                )
                -- SELECT * FROM pi_names;

                -- KEEP ONLY PAPERS FROM 2015 - 2019; denote all papers post 2017 as u19_period 
                ,papers_with_topics AS(
                SELECT DISTINCT
                       CASE WHEN pi_names.last_name IS NOT NULL THEN 1 ELSE 0 END AS is_pi,
                       budget_start,
                       complete_project_info.U19,
                       link_tables.pmid,
                       CONCAT(CONCAT(LEFT(affiliations.first_name,1),' '), affiliations.last_name) as name_key,
                       affiliations.first_name, 
                       affiliations.middle_name, 
                       affiliations.last_name, 
                       publications.pub_date,
                       publications.pub_date >= '2017-01-01' AS u19_period
                FROM complete_project_info
                JOIN link_tables   ON link_tables.project_number = complete_project_info.core_project_num
                JOIN publications  ON publications.pmid          = link_tables.pmid 
                JOIN affiliations  ON affiliations.pmid          = link_tables.pmid
                LEFT JOIN pi_names ON LEFT(pi_names.first_name,1) = LEFT(affiliations.first_name,1) and pi_names.last_name = affiliations.last_name
                WHERE publications.pub_date >= '2013-01-01' AND publications.pub_date <= '2020-12-31'
                ORDER BY U19 DESC, pub_date
                )

                -- SELECT * FROM papers_with_topics;
                ,subset as(
                SELECT *
                  FROM topics 
                 WHERE class = 'major'
                 AND pmid IN (SELECT DISTINCT pmid FROM papers_with_topics)
                )

                -- SELECT * FROM subset;
                ,number_of_topics as(
                SELECT pmid, COUNT(pmid) as num_topics FROM subset  GROUP BY pmid
                )

                -- SELECT * FROM number_of_topics;
                ,shared_topics AS(
                SELECT s.*, t.pmid as other_pmid, n.num_topics
                FROM subset s
                left join topics t ON t.topic_id = s.topic_id and t.pmid != s.pmid
                left join number_of_topics n on n.pmid = s.pmid
                WHERE (YEAR(t.pub_date) = YEAR(s.pub_date)) AND (MONTH(t.pub_date) = MONTH(s.pub_date)) -- MODIFY
                AND t.class = 'major'
                )

                ,topic_percentages AS(
                SELECT DISTINCT 
                pmid, 
                other_pmid, 
                100*COUNT(*)/num_topics as shared_topic_perc 
                -- COUNT(*) as shared_topic_perc 
                FROM shared_topics GROUP BY pmid,other_pmid
                )
                -- SELECT * FROM topic_percentages;

                , citation_count AS(
                SELECT citations.pmid, 
                       COUNT(citations.pmid) as times_cited
                FROM citations 
                WHERE citations.pmid IN ({core})
                GROUP BY citations.pmid
                )
                -- SELECT * FROM citation_count;

                ,citation_count_other AS(
                SELECT DISTINCT citations.pmid, 
                       COUNT(citations.pmid) as times_cited
                FROM citations
                WHERE citations.pmid IN ({other})
                GROUP BY citations.pmid
                )
                -- SELECT * FROM citation_count_other;

                ,relative_citations AS(
                SELECT DISTINCT s.pmid, s.other_pmid, s.pub_date, p.shared_topic_perc,
                       CASE WHEN c.times_cited IS NULL THEN 0 ELSE c.times_cited END as this_cited,
                       CASE WHEN o.times_cited IS NULL THEN 0 ELSE o.times_cited END as other_cited
                FROM shared_topics s
                LEFT JOIN citation_count c on c.pmid = s.pmid
                LEFT JOIN citation_count_other o on o.pmid = s.other_pmid
                LEFT JOIN topic_percentages p on p.other_pmid = s.other_pmid AND p.pmid = s.pmid
                )
                -- SELECT * FROM relative_citations;

                ,averages AS(
                SELECT pmid, 
                this_cited, 
                SUM( shared_topic_perc * other_cited ) / SUM( shared_topic_perc ) as other_weighted_avg,
                SUM(shared_topic_perc) as sum_shared_topic_perc,
                shared_topic_perc * ((other_cited - SUM(shared_topic_perc * other_cited)/SUM(shared_topic_perc)) * (other_cited - SUM(shared_topic_perc * other_cited)/SUM(shared_topic_perc)) ) as weighted_deviation_from_mean
                FROM relative_citations 
                GROUP BY pmid
                )
                -- SELECT * FROM averages;

                ,relative_citations_plus AS(
                SELECT r.*, a.other_weighted_avg, a.sum_shared_topic_perc, a.weighted_deviation_from_mean
                FROM relative_citations r
                JOIN averages a on a.pmid = r.pmid
                )

                ,stats AS(
                SELECT pmid,
                       this_cited, 
                       COUNT(pmid)           as reference_group_size,
                       AVG(other_cited)      as other_avg,
                       STD(other_cited)      as other_std,
                       other_weighted_avg    as other_avg_weighted, 
                       SQRT(SUM(weighted_deviation_from_mean)/SUM(shared_topic_perc)) AS other_std_weighted
                FROM relative_citations_plus r
                GROUP BY pmid
                )

                SELECT DISTINCT s.*,
                (s.this_cited - s.other_avg_weighted)/ s.other_std_weighted as zscore_weighted,
                (s.this_cited - s.other_avg)/ s.other_std as zscore,
                p.u19_period,
                p.U19
                FROM stats s
                LEFT JOIN papers_with_topics p ON p.pmid = s.pmid
                """
)

## 2. Compare Groups

In [ ]:
import pandas as pd
df = pd.DataFrame(stats)
df

In [71]:
# Find the median number of papers in the reference group.
ref_median = df['reference_group_size'].median()

# 2017 U19 Group, publication performance in 2013-2016
ind = (df["reference_group_size"] >= 10) & (df["u19_period"] == 0) & (df["U19"] == 2017)
print(df.loc[ind,"zscore"].mean())

# 2017 U19 Group, publication performance in 2017-2020
ind = (df["reference_group_size"] >= 10) & (df["u19_period"] == 1) & (df["U19"] == 2017)
print(df.loc[ind,"zscore"].mean())

# >2020 U19 Group, publication performance in 2013-2016
ind = (df["reference_group_size"] >= 10) & (df["u19_period"] == 0) & (df["U19"] > 2017)
print(df.loc[ind,"zscore"].mean())

# >2020 U19 Group, publication performance in 2013-2016
ind = (df["reference_group_size"] >= 10) & (df["u19_period"] == 1) & (df["U19"] > 2017)
print(df.loc[ind,"zscore"].mean())


2.7873839176321518
2.1232473737977124
1.5846800482290533
2.488643900377292
